In [2]:
import nbformat as nbf
import lxml.etree as etree
import sys
import requests
import os
import string
import hashlib
import urllib2
import ast

In [2]:
template = '/home/fbrito/fully_automated/src/main/app-resources/application_template.xml'

descriptor_file = '/home/fbrito/fully_automated/src/main/app-resources/application.xml'

nb_source = '/home/fbrito/fully_automated/src/main/app-resources/notebook/libexec/input.ipynb'

In [ ]:


nb = nbf.read(nb_source, 4)

tree = etree.parse(template)
root = tree.getroot()

for index, cell in enumerate(nb['cells']):
    print index
    
    if str(cell['cell_type']) == 'code': 
        
        try:
            root_ast = ast.parse(str(cell['source']))
            names = list({node.id for node in ast.walk(root_ast) if isinstance(node, ast.Name)})
            
            if len(names) != 2:
                continue
               
            if names[0] == 'input_reference' and action == 'runtime':
                el_source = root.xpath('/application/workflow/node/sources/source',
                                            namespaces={}) 
        
                el_source[0].text = str(cell['source'])
                    
            if names[0] == 'dict' or names[1] == 'dict':
                
                # deal with the alphabetical order
                if names[1] == 'dict': 
                    names[1] = names[0]
                    names[0] = 'dict'
                    
                print(str(cell['source']))   
                eval(str(cell['source']))
                
                if 'title' in eval(names[1]).keys() and 'abstract' in eval(names[1]).keys() and 'id' in eval(names[1]).keys():
                    
                    if 'value' in eval(names[1]).keys():
                        # it's a parameter
                        xml_string = '<parameter id="%s" title="%s" abstract="%s" maxOccurs="1" scope="runtime" type="LiteralData">%s</parameter>' % (eval(names[1])['id'], 
                                  eval(names[1])['title'], 
                                  eval(names[1])['abstract'], 
                                  eval(names[1])['value'])
                        
                        el_default_params = root.xpath('/application/jobTemplates/jobTemplate/defaultParameters', 
                                namespaces={})
                        
                        el_default_params[0].append(etree.fromstring(xml_string))
                     
                    else:
                       
                        # it's the service definition
                        el_workflow = root.xpath('/application/workflow',
                                            namespaces={}) 
        
                        el_workflow[0].attrib['id'] = eval(names[1])['id']
                        el_workflow[0].attrib['title'] = eval(names[1])['title']
                        el_workflow[0].attrib['abstract'] = eval(names[1])['abstract']
                    
        except SyntaxError:
            continue
          
descriptor = open(descriptor_file, 'wb')
descriptor.write(etree.tostring(tree, pretty_print=True))
descriptor.close()

In [48]:
nb['cells'][7]

{u'cell_type': u'code',
 u'execution_count': 10,
 u'metadata': {u'collapsed': True},
 u'outputs': [],
 u'source': u"input_identifier = 'S2A_MSIL2A_20170909T110651_N0205_R137_T29SPB_20170909T111217'"}

In [20]:
exec(str(nb['cells'][5]['source']))

In [22]:
type(a_par)

dict

In [23]:
type(pa_name)

dict

In [44]:
root_ast = ast.parse(str(nb['cells'][2]['source']))
names = list({node.id for node in ast.walk(root_ast) if isinstance(node, ast.Name)})
            

In [45]:
names

['dict', 'pa_code']

In [30]:
root_ast = ast.parse(str(nb['cells'][4]['source']))
names = {node.id for node in ast.walk(root_ast) if isinstance(node, ast.Name)}

In [31]:
names

{'dict', 'pa_name'}

In [49]:
len(names)

1